In [1]:
import sys
sys.path.append("./") 
from fastapi.testclient import TestClient
import api
import importlib as imp

import configparser

In [2]:
config = configparser.ConfigParser()
config.read("../../config.ini")
config.sections()

['DB']

In [3]:
imp.reload(api)
app = api.backendApi(dict(config['DB']))
client = TestClient(app.get_app())

In [7]:
response = client.get("/configuration")
print(response.json())

{'cities': [{'name': 'Atlanta', 'id': 1}], 'times_of_day': ['morning'], 'poi_categories': ['Schools and Kindergartners', 'Grocery stores and supermarkets', 'Cinemas and Theaters', 'Clinics and Hospitals', 'Restaurants', 'Vaccination centre'], 'demographic_categories': ['Race', 'Age and Sex', 'Income', 'Origin', 'Vehicle Availability']}


In [10]:
response = client.get("/pois/1/Restaurants")
print(response.json())

{'data': [{'id': 310449, 'name': 'Raging Burrito', 'h3id': '8944c106d3bffff', 'category': 'Restaurants', 'coords': {'lat': 33.774467, 'long': -84.29553}, 'lat': 33.774467, 'long': -84.29553}, {'id': 310451, 'name': 'Pho24 Vietnamese Noodle Soup', 'h3id': '8944c1a9a73ffff', 'category': 'Restaurants', 'coords': {'lat': 33.789627, 'long': -84.30708}, 'lat': 33.789627, 'long': -84.30708}, {'id': 310924, 'name': 'Thumbs Up Diner', 'h3id': '8944c1aa5b7ffff', 'category': 'Restaurants', 'coords': {'lat': 33.678883, 'long': -84.44167}, 'lat': 33.678883, 'long': -84.44167}, {'id': 310925, 'name': 'American Burger', 'h3id': '8944c1aa5b7ffff', 'category': 'Restaurants', 'coords': {'lat': 33.67881, 'long': -84.44139}, 'lat': 33.67881, 'long': -84.44139}, {'id': 310926, 'name': 'OZ Pizza', 'h3id': '8944c1aa5b3ffff', 'category': 'Restaurants', 'coords': {'lat': 33.678818, 'long': -84.44064}, 'lat': 33.678818, 'long': -84.44064}, {'id': 310932, 'name': 'Brake Pad', 'h3id': '8944c1b8ba7ffff', 'category

In [16]:
#To do: accessibility
response = client.get("/demographics?city=1&category=65 years and over female")
print(response.json())

{'detail': 'Not Found'}


In [17]:
response = client.get("/catchment?city=1&poi=1&TimeofDay=morning&distance=30&category=65 years and over female")
print(len(response.json()))

1
